In [1]:
# Start by importing the `tell` package and information about your operating system:
import os 
import tell

import numpy as np
import pandas as pd

from pandas import DataFrame


In [2]:
# Identify the current working directory, the subdirectory where the data will be stored, and the image output subdirectory:
current_dir =  os.path.join(os.path.dirname(os.getcwd()))
tell_data_dir = os.path.join(current_dir, r'tell_data')
tell_image_dir = os.path.join(tell_data_dir, r'visualizations')

# If the "tell_data_dir" subdirectory doesn't exist then create it:
if not os.path.exists(tell_data_dir):
   os.makedirs(tell_data_dir)

# If the "tell_image_dir" subdirectory doesn't exist then create it:
if not os.path.exists(tell_image_dir):
   os.makedirs(tell_image_dir)

data_input_dir = tell_data_dir

# Set the output directory based on the "raw_data_dir" variable:
output_dir = os.path.join(data_input_dir, r'tell_quickstarter_data', r'outputs', r'ba_service_territory')


In [7]:
def count_matches(states_key: str, fips_key: str, ignore=('and', 'if', 'on', 'an', 'a', 'the')) -> dict:
    """Count the number of word matches between two primary keys.

    :param states_key:                 Key of the <state_abbrev>_<county_name> in the 'df_states' dataframe
    :type states_key:                  str

    :param fips_key:                   Key of the <state_abbrev>_<county_name> in the 'df_fips' dataframe
    :type fips_key:                    str

    :param ignore:                     A list of common english words to ignore
    :type ignore:                      list

    :return:                           Total number of matches or None if there are no matches

    """

    # Split the states names from the 'states_key' list:
    states_split = states_key.split('_')
    states_state_name = states_split[0]

    # Split the FIPS code from the 'fips_key' list:
    fips_split = fips_key.split('_')
    fips_state_name = fips_split[0]

    # If the state names match then proceed, else return None:
    if states_state_name == fips_state_name:

        # Split out state abbreviation and space separator and remove underscores:
        states_key_split = [ix[0] for ix in [i.split('-') for i in ' '.join(states_split[1:]).split(' ')]]

        # Check for any "de" suffix and join to the second position (e.g., change "de witt" to "dewitt"):
        states_key_split = combine_elements('de', states_key_split)

        # Set initial the count of matches to zero:
        ct = 0

        # Loop over each word in the 'states_key' and count the matches:
        for word in states_key_split:

            # Split out state abbreviation and space separator and remove underscores:
            fips_key_split = [ix[0] for ix in [i.split('-') for i in ' '.join(fips_split[1:]).split(' ')]]

            # Check for any "de" suffix and join to the second position (e.g., change "de witt" to "dewitt"):
            fips_key_split = combine_elements('de', fips_key_split)

            # If the word matches the 'fips_key' list then increment the counter:
            if (word not in ignore) and (word in fips_key_split):
               ct += 1

        # Return the number of matches or, if there were no matches, return None:
        if ct > 0:
            return {fips_key: ct}
        else:
            return None

    else:
        return None
    

In [9]:
def combine_elements(part: str, part_list: list) -> list:
    """Check for any suffixes and join them to the second position (e.g., change "de witt" to "dewitt")

    :param part:                The suffix of interest to be searched for in the parts_list
    :type part:                 str

    :param part_list:           The string of interest separated into a list of parts
    :type part_list:            list

    :return:                    Joined part list with combined to suffix

    """

    # I don't entirely understand what is happening in this code block:
    if part in part_list:
        one_idx = part_list.index(part)
        combined = f"{part_list[one_idx]}{part_list[one_idx + 1]}"
        part_list.pop(one_idx + 1)
        part_list.pop(one_idx)
        part_list.insert(0, combined)

    return part_list

In [10]:
def keep_valid(x: dict) -> dict:
    """Keep only dictionaries that have a non-zero count for a county name present.

    :param x:                Dictionary with matches from filter_two
    :type x:                 dict

    :return:                 Dictionary with non-zero count of potential county names

    """

    # Initiate an empty dictionary to store the results:
    d = {}

    # Loop over the dictionary of matches from filter two:
    for value in x:
        # If there is a non-zero count of potential matches then extract that row:
        if type(value) is dict:
            key = list(value.keys())[0]
            d[key] = value[key]

    # Return the dictionary of valid matches:
    return d


In [11]:
# Can Casey M. or somebody please comment this code block:
def find_county(d: dict) -> dict:
    """Add the FIPS code to the data frame where the optimal value (i.e., >= 1) has been identified.

    :param d:              Dictionary with non-zero count of potential county names
    :type d:               dict

    :return:               Dictionary of FIPS codes combined to the county dictionary with a count of 1 or more
      
    """

    # I don't entirely understand what is happening in this code block:
    if len(d) > 0:
        values = [d[k] for k in d.keys()]
        max_value = max(values)

        keys, count = np.unique(values, return_counts=True)
        val_dict = {}

        for k, v in zip(keys, count):
            val_dict[k] = v

        if val_dict[max_value] > 1:
            return None
        else:
            return list(d.keys())[list(d.values()).index(max_value)]

    else:
        return None

In [12]:
# Can Casey M. or somebody please comment this code block:
def get_max_count(d: dict) -> DataFrame:
    """Count of the optimal county name matches: Ideally this is 1, but if it's different then investigate further

     :param d:              Dictionary of FIPS codes combined to the county dictionary with a count of 1 or more
     :type d:               dict

     :return:               DataFrame of FIPS codes with column count of optimal county name

     """

    # I don't entirely understand what is happening in this code block:
    if len(d) > 0:

        values = [d[k] for k in d.keys()]

        max_value = max(values)

        keys, count = np.unique(values, return_counts=True)
        val_dict = {}

        for k, v in zip(keys, count):
            val_dict[k] = v

        return val_dict[max_value]

    else:
        return None

In [3]:
# Set the target year:
target_year = 2019

# Set paths to files:
fips_file = os.path.join(data_input_dir, r'tell_raw_data', 'state_and_county_fips_codes.csv')
service_area_file = os.path.join(data_input_dir, r'tell_raw_data', r'EIA_861', f'{target_year}', f'Service_Territory_{target_year}.xlsx')
sales_ult_file = os.path.join(data_input_dir, r'tell_raw_data', r'EIA_861', f'{target_year}', f'Sales_Ult_Cust_{target_year}.xlsx')
bal_auth_file = os.path.join(data_input_dir, r'tell_raw_data', r'EIA_861', f'{target_year}', f'Balancing_Authority_{target_year}.xlsx')


In [4]:
# Run the "prepare_data" function:

# Read in the state and county FIPS code .csv file:
df_fips = pd.read_csv(fips_file)

# Read in the raw data from the EIA-861 Excel spreadsheets:
try:
    df_states = pd.read_excel(service_area_file, sheet_name='Counties')
except:
    df_states = pd.read_excel(service_area_file, sheet_name='Counties_States')
df_ult = pd.read_excel(sales_ult_file, sheet_name='States', skiprows=2)
df_ba = pd.read_excel(bal_auth_file)

# Strip the word "county" name from full reference and make it consistently lower case:
df_fips['county_lower'] = df_fips['county_name'].apply(lambda x: x.lower().split(' county')[0])

# Replace apostrophes:
df_fips['county_lower'] = df_fips['county_lower'].apply(lambda x: x.lower().split(' parish')[0]).str.replace("'", "")

# Make the state abbreviations and state names lower case:
df_fips['state_abbreviation'] = df_fips['state_abbreviation'].str.lower()

# Make the state names lower case:
df_states['State'] = df_states['State'].str.lower()

# Replace apostrophes:
df_states['county_lower'] = df_states['County'].str.lower().str.replace("'", "")

# Create a unified <state_abbrev>_<county_lower> key to merge by:
df_fips['fips_key'] = df_fips['state_abbreviation'] + '_' + df_fips['county_lower']
df_states['states_key'] = df_states['State'] + '_' + df_states['county_lower']

# Filter the "df_ult" and "df_ba" dataframes to only the columns we need:
try:
    df_ult = df_ult[["Utility Number", "Utility Name", "BA_CODE"]].rename(columns={'BA_CODE': 'BA Code'})
except:
    df_ult = df_ult[["Utility Number", "Utility Name", "BA Code"]]
df_ba = df_ba[["BA Code", "BA ID", "Balancing Authority Name"]]


In [5]:
# Run the "filter_one" function:

# Merge the 'df_states' and 'df_fips' dataframes based on the common key:
df_states_fips = pd.merge(left=df_states, right=df_fips, left_on='states_key', right_on='fips_key', how='left')

# Reassign a single variable as a float:
df_states_fips['Utility Number'] = df_states_fips['Utility Number'].astype(float)

# Merge the 'df_states_fips' and 'df_ult' dataframes based on the utility number:
df_fips_ult = df_states_fips.merge(df_ult, on='Utility Number', how='left')

# Merge the 'df_fips_ult' and 'df_ba' dataframes based on the BA code:
df_valid = df_fips_ult.merge(df_ba, left_on='BA Code', right_on='BA Code', how='left')

# Filter out rows that did not have a valid match:
df_nan = df_valid.loc[df_valid['county_lower_y'].isna()].copy()

# Drop the rows that did not have a valid match:
df_valid = df_valid.loc[~df_valid['county_lower_y'].isna()].copy()

# Drop unneeded columns from the valid and invalid dataframes:
df_valid.drop(columns=['county_lower_y', 'Utility Name_y'], inplace=True)
df_nan.drop(columns=['county_lower_y'], inplace=True)

df_fips_ult


,Data Year,Utility Number,Utility Name_x,Short Form,State,County,county_lower_x,states_key,state_name,state_abbreviation,state_FIPS,county_name,county_FIPS,county_lower_y,fips_key,Utility Name_y,BA Code
0,2019,34.0,City of Abbeville - (SC),Y,sc,Abbeville,abbeville,sc_abbeville,South Carolina,sc,45000.0,Abbeville County,45001.0,abbeville,sc_abbeville,City of Abbeville - (SC),DUK
1,2019,55.0,City of Aberdeen - (MS),NaN,ms,Monroe,monroe,ms_monroe,Mississippi,ms,28000.0,Monroe County,28095.0,monroe,ms_monroe,City of Aberdeen - (MS),TVA
2,2019,59.0,City of Abbeville - (LA),NaN,la,Vermilion,vermilion,la_vermilion,Louisiana,la,22000.0,Vermilion Parish,22113.0,vermilion,la_vermilion,City of Abbeville - (LA),MISO
3,2019,84.0,A & N Electric Coop,NaN,md,Somerset,somerset,md_somerset,Maryland,md,24000.0,Somerset County,24039.0,somerset,md_somerset,A & N Electric Coop,PJM
4,2019,84.0,A & N Electric Coop,NaN,md,Somerset,somerset,md_somerset,Maryland,md,24000.0,Somerset County,24039.0,somerset,md_somerset,A & N Electric Coop,PJM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19448,2019,61432.0,Monterey Bay Community Power,NaN,ca,Monterey,monterey,ca_monterey,California,ca,6000.0,Monterey County,6053.0,monterey,ca_monterey,Monterey Bay Community Power,CISO
19449,2019,61432.0,Monterey Bay Community Power,NaN,ca,San Benito,san benito,ca_san benito,California,ca,6000.0,San Benito County,6069.0,san benito,ca_san benito,Monterey Bay Community Power,CISO
19450,2019,61432.0,Monterey Bay Community Power,NaN,ca,Santa Cruz,santa cruz,ca_santa cruz,California,ca,6000.0,Santa Cruz County,6087.0,santa cruz,ca_santa cruz,Monterey Bay Community Power,CISO
19451,2019,62107.0,City of Industry,NaN,ca,Los Angeles,los angeles,ca_los angeles,California,ca,6000.0,Los Angeles County,6037.0,los angeles,ca_los angeles,City of Industry,CISO


In [16]:
# Run the "filter_two" function:

# Get keys from states that are in the "df_fips" dataframe that have NaN records:
nan_keys_fips = df_fips.loc[df_fips['state_abbreviation'].isin(df_nan['State'].unique())]['fips_key'].unique()

# Extract the possible matches from the "df_nan" dataframe:
df_nan['matches'] = [[]] * df_nan.shape[0]

# Loop over the invalid keys and count the matches using the "count_matches" function:
for index, fips_key in enumerate(nan_keys_fips):
    df_nan['matches'] = df_nan['matches'] + df_nan['states_key'].apply(lambda x: [count_matches(x, fips_key)])

# Keep only dictionaries that have a non-zero count for a county name present using the "keep_valid" function:
df_nan['matches'] = df_nan['matches'].apply(keep_valid)

# Add a new column with the maximum possible county name matches sing the "get_max_count" function (ideally this is 1):
df_nan['count_of_selected'] = df_nan['matches'].apply(get_max_count)

# Add the FIPS key to the dataframe using the "find_county" function:
df_nan['fips_key'] = df_nan['matches'].apply(find_county)

# Drop columns we no longer need:
df_nan.drop(columns=['state_name', 'state_abbreviation', 'state_FIPS', 'county_name', 'county_FIPS'], inplace=True)

# Extract rows that now have a vaild match:
df_nan_good = df_nan.loc[df_nan['count_of_selected'] == 1].copy()

# Extract rows that still have an invalid match:
df_nan_bad = df_nan.loc[df_nan['count_of_selected'] != 1].copy()

# Merge the valid match dataframes together using the FIPS code:
mrx = pd.merge(left=df_nan_good, right=df_fips, left_on='fips_key', right_on='fips_key', how='left')

# Drop columns we no longer need:
mrx.drop(columns=['count_of_selected', 'matches'], inplace=True)

# Reorder the columns to match the pre-existing "df_valid" dataframe:
mrx = mrx[df_valid.columns].copy()

# Concatenate the two dataframes with valid matches together:
df_valid = pd.concat([df_valid, mrx])

df_valid

KeyError: "['state_name', 'state_abbreviation', 'state_FIPS', 'county_name', 'county_FIPS'] not found in axis"

In [15]:
# Run the "data_format" section:

# Establish the column names we want to keep:
col_names = ['Data Year', 'Utility Number', 'Utility Name_x', 'state_abbreviation', 'state_name',
             'state_FIPS', 'county_name', 'county_FIPS', 'BA ID', 'BA Code',
             'Balancing Authority Name']

# Only keep the columns that are needed:
df = df[col_names].copy()

# Rename the columns using a consistent format:
df.rename(columns={"Data Year": "year",
                   "Utility Number": "utility_number",
                   "Utility Name_x": "utility_name",
                   "state_abbreviation": "state_abbreviation",
                   "state_name": "state_name",
                   "state_FIPS": "state_fips",
                   "county_name": "county_name",
                   "county_FIPS": "county_fips",
                   "BA ID": "ba_number",
                   "BA Code": "ba_abbreviation",
                   "Balancing Authority Name": "ba_name"}, inplace=True)

# Remove commas from the 'ba_name' variable:
df['ba_name'] = df['ba_name'].str.replace(",", "")

# Keep only the variables that are used later in tell:
df = df[['year',
         'state_fips',
         'state_name',
         'county_fips',
         'county_name',
         'ba_number',
         'ba_abbreviation']].copy(deep=False)

# Rename the columns to use leading capital letters:
df.rename(columns={"year": "Year",
                   "state_fips": "State_FIPS",
                   "state_name": "State_Name",
                   "county_fips": "County_FIPS",
                   "county_name": "County_Name",
                   "ba_number": "BA_Number",
                   "ba_abbreviation": "BA_Code"}, inplace=True)


NameError: name 'df' is not defined